### ElemNet Installation

ElemNet is a deep neural network model that takes only the elemental compositions as inputs and leverages artificial intelligence to automatically capture the essential chemistry to predict materials properties. ElemNet can automatically learn the chemical interactions and similarities between different elements which allows it to even predict the phase diagrams of chemical systems absent from the training dataset more accurately than the conventional machine learning models based on physical attributes levaraging domain knowledge.

    https://github.com/NU-CUCIS/ElemNet/blob/master/README.md
    
#### Installation requirements
The basic requirement for re-using these environments are a Python 3.6.3 Jupyter environment with the packages listed in requirements.txt.

Some analyses required the use of Magpie, which requires Java JDK 1.7 or greater. See [the Magpie documentation for details].



In [1]:
# Requirements:
import pandas as pd, warnings, sklearn, numpy as np, matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
#Conda env: Scipy, pymatgen, matminer (runs w/ lower pandas), tensorflow 